In [1]:
from __future__ import print_function

When analyzing data, I usually use the following three modules. I use pandas for data management, filtering, grouping, and processing. I use numpy for basic array math. I use toyplot for rendering the charts.

In [2]:
import pandas
import numpy
import toyplot
import toyplot.pdf
import toyplot.png
import toyplot.svg

print('Pandas version:  ', pandas.__version__)
print('Numpy version:   ', numpy.__version__)
print('Toyplot version: ', toyplot.__version__)

Pandas version:   0.19.2
Numpy version:    1.12.0
Toyplot version:  0.14.0-dev


Load in the "auto" dataset. This is a fun collection of data on cars manufactured between 1970 and 1982. The source for this data can be found at https://archive.ics.uci.edu/ml/datasets/Auto+MPG.

The data are stored in a text file containing columns of data. We use the pandas.read_table() method to parse the data and load it in a pandas DataFrame. The file does not contain a header row, so we need to specify the names of the columns manually.

In [3]:
column_names = ['MPG',
                'Cylinders',
                'Displacement',
                'Horsepower',
                'Weight',
                'Acceleration',
                'Model Year',
                'Origin Index',
                'Car Name']
data = pandas.read_table('auto-mpg.data',
                         delim_whitespace=True,
                         names=column_names,
                         index_col=False)

The origin column indicates the country of origin for the car manufacture. It has three numeric values, 1, 2, or 3. These indicate USA, Europe, or Japan, respectively. Replace the origin column with a string representing the country name.

In [4]:
country_map = pandas.Series(index=[1,2,3],
                            data=['USA', 'Europe', 'Japan'])
data['Origin'] = numpy.array(country_map[data['Origin Index']])

In this plot we are going to show the trend of the average miles per gallon (MPG) rating for subsequent model years separated by country of origin. This time period saw a significant increase in MPG driven by the U.S. fuel crisis. We can use the pivot_table feature of pandas to get this information from the data. (Excel and other spreadsheets have similar functionality.)

In [5]:
average_mpg_per_year = data.pivot_table(index='Model Year',
                                        columns='Origin',
                                        values='MPG',
                                        aggfunc='mean')
average_mpg_per_year

Origin,Europe,Japan,USA
Model Year,,,
70,25.200000,25.500000,15.272727
71,28.750000,29.500000,18.100000
72,22.000000,24.200000,16.277778
73,24.000000,20.000000,15.034483
74,27.000000,29.333333,18.333333
75,24.500000,27.500000,17.550000
76,24.250000,28.000000,19.431818
77,29.250000,27.416667,20.722222
78,24.950000,29.687500,21.772727


Use toyplot to make a plot of the MPG for every car in the database organized by year and colored by origin.

In [6]:
canvas = toyplot.Canvas('4in', '2.6in')

axes = canvas.cartesian(bounds=(41,-1,6,-43),
                        xlabel = 'Model Year',
                        ylabel = 'MPG')

colormap = toyplot.color.CategoricalMap()

axes.scatterplot(data['Model Year'] + 1900 + 0.2*(data['Origin Index']-2),
                 data['MPG'],
                 size=4,
                 opacity=0.75,
                 color=(numpy.array(data['Origin Index'])-1,colormap))

for country in country_map:
    series = average_mpg_per_year[country]
    x = series.index[-1] + 1900
    y = numpy.array(series)[-1]
    axes.text(x, y, country,
              style={"text-anchor":"start",
                     "-toyplot-anchor-shift":"15px"})

# It's usually best to make the y-axis 0-based.
axes.y.domain.min = 0

# Toyplot is sometimes inaccurate in judging the width of labels.
axes.x.domain.max = 1984.2

# The labels can make for odd tick placement.
# Place them manually
axes.x.ticks.locator = \
    toyplot.locator.Explicit([1970,1974,1978,1982])

<svg class="toyplot-canvas-Canvas" height="249.60000000000002px" id="t5f90f7d7032540b88883220edd5b7f7d" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 384.0 249.60000000000002" width="384.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> USA Europe Japan 1970 1974 1978 1982 Model Year 0 10 20 30 40 50 MPG 
 
 
 Save as .csv

In [7]:
toyplot.pdf.render(canvas, 'Detail_MultiSeries.pdf')
toyplot.svg.render(canvas, 'Detail_MultiSeries.svg')
toyplot.png.render(canvas, 'Detail_MultiSeries.png', scale=5)

Now use toyplot to plot this data along with trend lines.

In [8]:
canvas = toyplot.Canvas('4in', '2.6in')

axes = canvas.cartesian(bounds=(41,-1,6,-43),
                        xlabel = 'Model Year',
                        ylabel = 'MPG')

colormap = toyplot.color.CategoricalMap()

axes.scatterplot(data['Model Year'] + 1900 + 0.2*(data['Origin Index']-2),
                 data['MPG'],
                 size=4,
                 opacity=1.0,
                 color=(numpy.array(data['Origin Index'])-1,colormap))

for column in country_map:
    series = average_mpg_per_year[column]
    x = series.index + 1900
    y = numpy.array(series)
    axes.plot(x, y, opacity=0.5)
    axes.text(x[-1], y[-1], column,
              style={"text-anchor":"start",
                     "-toyplot-anchor-shift":"10px"})

# It's usually best to make the y-axis 0-based.
axes.y.domain.min = 0

# Toyplot is sometimes inaccurate in judging the width of labels.
axes.x.domain.max = 1984.2

# The labels can make for odd tick placement.
# Place them manually
axes.x.ticks.locator = \
    toyplot.locator.Explicit([1970,1974,1978,1982])

<svg class="toyplot-canvas-Canvas" height="249.60000000000002px" id="ta0825a5b385048fa8027b1629a7c22ed" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 384.0 249.60000000000002" width="384.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <path d="M 45.750000000001052 145.32581818181822 L 69.500000000000909 133.9828 L 93.250000000000753 141.29355555555557 L 117.0000000000006 146.28165517241379 L 140.75000000000045 133.04666666666665 L 164.50000000000028 136.18940000000001 L 188.25000000000014 128.63954545454547 L 212.0 123.46244444444447 L 235.74999999999983 119.24781818181818 L 259.49999999999972 112.40521739130436 L 283.24999999999955 102.63188571428574 L 306.99999999999943 96.146553846153822 L 330.7499999999992 88.446600000000018" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:0.5;stroke-width:2.0"> USA <path d="M 45.750000000001052 105.49760000000001 L 69.500000000000909 91.255000000000024 L 93.250000000000753 118.33600000000003 L 117.0000000000006 110.31200000000001 L 140.75000000000045 98.275999999999996 L 164.50000000000028 108.30600000000001 L 188.25000000000014 109.30900000000001 L 212.0 89.249000000000024 L 235.74999999999983 106.50060000000001 L 259.49999999999972 84.434600000000003 L 283.24999999999955 56.996977777777758 L 306.99999999999943 79.921100000000024 L 330.7499999999992 46.11999999999999" style="stroke:rgb(98.8%,55.3%,38.4%);stroke-opacity:0.5;stroke-width:2.0"> Europe <path d="M 45.750000000001052 104.29400000000001 L 69.500000000000909 88.246000000000024 L 93.250000000000753 109.50960000000001 L 117.0000000000006 126.36000000000001 L 140.75000000000045 88.914666666666676 L 164.50000000000028 96.269999999999996 L 188.25000000000014 94.263999999999996 L 212.0 96.604333333333344 L 235.74999999999983 87.493750000000034 L 259.49999999999972 74.404600000000002 L 283.24999999999955 64.575200000000009 L 306.99999999999943 74.371166666666696 L 330.7499999999992 66.625777777777799" style="stroke:rgb(55.3%,62.7%,79.6%);stroke-opacity:0.5;stroke-width:2.0"> Japan 1970 1974 1978 1982 Model Year 0 10 20 30 40 50 MPG 
 
 
 Save as .csv

In [9]:
toyplot.pdf.render(canvas, 'Detail_MultiSeries_Trend.pdf')
toyplot.svg.render(canvas, 'Detail_MultiSeries_Trend.svg')
toyplot.png.render(canvas, 'Detail_MultiSeries_Trend.png', scale=5)